#   Homework 3
## Sentiment analysis using Neural Networks

Total: 50 Points


In this homework we will perform sentiment analysis using a few simple Neural Network based architectures.
For this problem we use the IMDB Large Movie Review Dataset. The dataset contains 25,000 highly polar movie reviews for both train and test dataset, each with 12,500 positive (greater than equal to 7/10 rating) and 12,500 negative reviews(less than equal to 4/10 rating). 

Use "https://keras.io/" for keras documentation. Please use Python 3. GPU is not required but it will help improve the training speed for each problem.

Please save the notebook with your cell outputs. You will not be graded if your outputs are not present below the homework cell. Also note your outputs will be unique since you will be using your the last numbers of your uni as your random seed (In the third cell). Make sure you submit this iPython file, with the saved outputs. The submission format must be 'hw3/hw3.ipynb'. You will not submit any other files. If you do save your model weights, you will not submit them. You will however, make sure your model weights do get saved in the 'weights' folder and can be retrieved from there as well.

Please fill your details below.



Name: Haiqi Zhu

Uni: hz2432

Email: hz2432@columbia.edu


In [1]:
from os import listdir
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution1D, GRU, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
#from keras import initializers
from keras import backend as K
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras import optimizers
import numpy as np

Using TensorFlow backend.


In [2]:


#we retrieve train and test file names

train_dir = "./aclImdb/train/"
test_dir = "./aclImdb/test/"
tr_review = [re_filename for re_filename in listdir(train_dir)]
te_review = [re_filename for re_filename in listdir(test_dir)]

#we initialize the train and test arrays

tr_X = []
tr_Y = []
te_X = []
te_Y = []

#we arrange the reviews into the train and test arrays 

for review_file in tr_review:
    f_review = open(train_dir+review_file, "r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    tr_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        tr_Y.append(1)
    else:
        tr_Y.append(0)
        
for review_file in te_review:
    f_review = open(test_dir+review_file, "r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    te_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        te_Y.append(1)
    else:
        te_Y.append(0)
        

We will now create the validation set from the train set

use the last 4 numbers of your uni for the seed value seed to ensure all answers remain unique.

In [3]:
#replace 2 (SEED) with the last 4 numbers of your Uni
#Uni: hz2432
SEED = 2432
seed_counter = 0
while(1):

    shuffle_combine = list(zip(tr_X, tr_Y))
    random.seed(SEED+seed_counter)
    seed_counter+=1
    random.shuffle(shuffle_combine)

    tr_X, tr_Y = zip(*shuffle_combine)

    val_X = tr_X[:5000]
    val_Y = tr_Y[:5000]

    counter = 0
    for label in val_Y:
        counter+=label

    print (counter)
    print (seed_counter)
    if(counter>2400 and counter <2600):
        tr_X = tr_X[5000:]
        tr_Y = tr_Y[5000:]
        break

2547
1


In [4]:


print("Length of Train review set : " + str(len(tr_X)))
print("Length of Train label set : " + str(len(tr_Y)))
print("Length of Validation review set : " + str(len(val_X)))
print("Length of Validation label set : " + str(len(val_Y)))
print("Length of Test review set : " + str(len(te_X)))
print("Length of Test label set : " + str(len(te_Y)))
print("*****************************************")
print("Some sample Reviews Train sets and their labels")
print(tr_X[0][:150])
print(tr_Y[0])
print(tr_X[1][:150])
print(tr_Y[1])
print(tr_X[2][:150])
print(tr_Y[2])
print(tr_X[3][:150])
print(tr_Y[3])
print(tr_X[4][:150])
print(tr_Y[4])

Length of Train review set : 20000
Length of Train label set : 20000
Length of Validation review set : 5000
Length of Validation label set : 5000
Length of Test review set : 25000
Length of Test label set : 25000
*****************************************
Some sample Reviews Train sets and their labels
I've recently watched this movie, in a lazy Sunday afternoon, with some friend of mine and we have a lot of fun! This movie is a masterpiece of trash.
1
Repugnant Bronson thriller. Unfortunately, it's technically good and I gave it 4/10, but it's so utterly vile that it would be inconceivable to call i
0
This film has so little class in comparison to Strangers on a Train or even, Accidental Meeting for that matter, that despite plot similarities I woul
0
"Jared Diamond made a point in the first episode that other peoples of the world didn't have animals to domesticate but Europeans did, and that accoun
1
By 1945, and after a string of solid WWII propaganda pieces, Errol Flynn’s hold over 

In [5]:
#we collect all the reviews from train validation and test set to generate 
texts = []
texts += tr_X 
texts += te_X 
texts += val_X
len(texts)



#we clip the sentence length to first 250 words. 
MAX_SEQUENCE_LENGTH = 250

#length of vocab, Tokenizer will only use vocab_len most common words
vocab_len = 25000

#we tokenize the texts and convert all the words to tokens
tokenizer = Tokenizer(num_words=vocab_len)
tokenizer.fit_on_texts(texts)

token_tr_X = tokenizer.texts_to_sequences(tr_X)
token_te_X = tokenizer.texts_to_sequences(te_X)
token_val_X = tokenizer.texts_to_sequences(val_X)

#to ensure all reviews have the same length, we pad the smaller reviews with 0, 
#and cut the larger reviews to a max length 
#(we clip from the top, as the end of the reviews generally have a conclusion which provides better features)
x_train = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)
x_test = sequence.pad_sequences(token_te_X, maxlen=MAX_SEQUENCE_LENGTH)
x_val = sequence.pad_sequences(token_val_X, maxlen=MAX_SEQUENCE_LENGTH)


#changes the labels to one-hot encoding
y_train = np_utils.to_categorical(tr_Y)
y_test = np_utils.to_categorical(te_Y)
y_val = np_utils.to_categorical(val_Y)


In [6]:
print('X_train shape:', x_train.shape)
print('X_test shape:', x_test.shape)
print('X_val shape:', x_val.shape)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print('y_val shape:', y_val.shape)


print("*****************************************")
print("Tokenized Reviews Train sets and their labels")
print(x_train[0][:20])
print(y_train[0])
print()
print(x_train[1][:20])
print(y_train[1])
print()
print(x_train[2][:20])
print(y_train[2])
print()
print(x_train[3][:20])
print(y_train[3])
print()
print(x_train[4][:20])
print(y_train[4])
print()

X_train shape: (20000, 250)
X_test shape: (25000, 250)
X_val shape: (5000, 250)
y_train shape: (20000, 2)
y_test shape: (25000, 2)
y_val shape: (5000, 2)
*****************************************
Tokenized Reviews Train sets and their labels
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1.  0.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1.  0.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]

[ 9499  2518     6    48     6    29    42    14    10   301   952   450
    29 16742   233    21    26     1  1950  6274]
[ 0.  1.]



********************************************

As you can see the reviews have now been transformed into indices to tokenized vocabulary and the labels have been converted to one-hot encoding. We can now go ahead and feed these sequences to Neural Network Models.

********************************************

# Part A

Building your first model (5 Points)

Construct this sequential model using Keras :

![title](img/model1.jpg)

In [7]:
print('Build model...')

## implement model here

model = Sequential()

model.add(Embedding(vocab_len, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Flatten())
model.add(Dense(200,activation="relu"))
model.add(Dense(2,activation="softmax"))

## compille it here according to instructions
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               6400200   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 9,600,602
Trainable params: 9,600,602
Non-trainable params: 0
_________________________________________________________________
Model Built


In [8]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 48s 2ms/step - loss: 0.4097 - acc: 0.7951 - val_loss: 0.2984 - val_acc: 0.8748
Epoch 2/4
20000/20000 [==============================] - 48s 2ms/step - loss: 0.0479 - acc: 0.9854 - val_loss: 0.4392 - val_acc: 0.8532
Epoch 3/4
20000/20000 [==============================] - 48s 2ms/step - loss: 0.0026 - acc: 0.9996 - val_loss: 0.5540 - val_acc: 0.8660
Epoch 4/4
20000/20000 [==============================] - 48s 2ms/step - loss: 9.4813e-05 - acc: 1.0000 - val_loss: 0.5664 - val_acc: 0.8682


# Part B

Stacking Fully Connected Layers (5 points)

Construct this sequential model using Keras :

![title](img/model2.jpg)

In [14]:
print('Build model...')

## implement model here

model = Sequential()

model.add(Embedding(vocab_len, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Flatten())
model.add(Dense(200,activation="relu"))
model.add(Dense(200,activation="relu"))
model.add(Dense(2,activation="softmax"))

## compille it here according to instructions
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 200)               6400200   
_________________________________________________________________
dense_6 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 402       
Total params: 9,640,802
Trainable params: 9,640,802
Non-trainable params: 0
_________________________________________________________________
Model Built


In [13]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 59s 3ms/step - loss: 0.3997 - acc: 0.8044 - val_loss: 0.2950 - val_acc: 0.8758
Epoch 2/4
20000/20000 [==============================] - 59s 3ms/step - loss: 0.0425 - acc: 0.9861 - val_loss: 0.5775 - val_acc: 0.8476
Epoch 3/4
20000/20000 [==============================] - 60s 3ms/step - loss: 0.0102 - acc: 0.9965 - val_loss: 0.6674 - val_acc: 0.8414
Epoch 4/4
20000/20000 [==============================] - 60s 3ms/step - loss: 0.0115 - acc: 0.9958 - val_loss: 0.7365 - val_acc: 0.8520


# Part C

Using LSTMS based networks(5 Points) 

Construct this sequential model using Keras :

![title](img/model3.jpg)

In [15]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(vocab_len, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(128))
model.add(Dense(128,activation="relu"))
model.add(Dense(2,activation="softmax"))

## compille it here according to instructions

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 258       
Total params: 3,348,354
Trainable params: 3,348,354
Non-trainable params: 0
_________________________________________________________________
Model Built


In [15]:

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 170s 9ms/step - loss: 0.4442 - acc: 0.7874 - val_loss: 0.3414 - val_acc: 0.8622
Epoch 2/5
20000/20000 [==============================] - 169s 8ms/step - loss: 0.2228 - acc: 0.9147 - val_loss: 0.4139 - val_acc: 0.8150
Epoch 3/5
20000/20000 [==============================] - 169s 8ms/step - loss: 0.1367 - acc: 0.9495 - val_loss: 0.4158 - val_acc: 0.8608
Epoch 4/5
20000/20000 [==============================] - 170s 9ms/step - loss: 0.0812 - acc: 0.9725 - val_loss: 0.4831 - val_acc: 0.8570
Epoch 5/5
20000/20000 [==============================] - 169s 8ms/step - loss: 0.0483 - acc: 0.9845 - val_loss: 0.5379 - val_acc: 0.8538


# Part D

Adding Pretrained Word Embeddings(10 Points)

Construct this sequential model using Keras :

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model4.jpg)

In [16]:
import codecs

#dimension of Glove Embeddings.
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

#load glove embeddings
gembeddings_index = {}
with codecs.open('glove.42B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

# nb_words contains the total length of vocab
nb_words = len(word_index) +1

#get glove embeddings for each word in tokenizer.
#g_word_embedding_matrix holds the embeddings dictionary
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
#total words in the tokenizer not in Embedding matrix
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))



Found 124252 unique tokens
G Word embeddings: 1917494
G Null word embeddings: 35772


In [13]:
print('Build model...')

## implement model here

model = Sequential()

model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM,
                            weights=[g_word_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False))
model.add(LSTM(128,recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(128,activation="relu"))
model.add(Dense(2,activation="softmax"))

## to use the glove embeddings, your embedding layer would take the vocab size as input dimension, 
## Glove embedding dimension as the output dimsion
## and you will provide the  embedding dictionary as the 'weights' parameter (!important) to the embedding layer.


## compille it here according to instructions

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 258       
Total params: 37,512,318
Trainable params: 236,418
Non-trainable params: 37,275,900
_________________________________________________________________
Model Built


In [29]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 216s 11ms/step - loss: 0.5227 - acc: 0.7303 - val_loss: 0.3382 - val_acc: 0.8586
Epoch 2/5
20000/20000 [==============================] - 216s 11ms/step - loss: 0.3123 - acc: 0.8690 - val_loss: 0.2811 - val_acc: 0.8836
Epoch 3/5
20000/20000 [==============================] - 216s 11ms/step - loss: 0.2570 - acc: 0.8953 - val_loss: 0.2652 - val_acc: 0.8918
Epoch 4/5
20000/20000 [==============================] - 216s 11ms/step - loss: 0.2241 - acc: 0.9114 - val_loss: 0.2569 - val_acc: 0.8942
Epoch 5/5
20000/20000 [==============================] - 216s 11ms/step - loss: 0.1894 - acc: 0.9283 - val_loss: 0.2609 - val_acc: 0.8952


# Dont attempt this

Stacking LSTM layers

Unfortunately it takes very long to train, be aware we can stack LTMSs over each other like this.
This requires bottom LSTM to return a sequences instead instead of single vector, which becomes input for the top LSTM.


![title](img/model5.jpg)

# Part E

Using Convolutional Networks (10 points)

Construct the model, shown below. Use the same loss functions and optimizers as before

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model6.jpg)

In [17]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[g_word_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False))

model.add(Convolution1D(128, 3,activation="relu"))
model.add(Dropout(0.2))
model.add(Convolution1D(64, 3,activation="relu"))
model.add(Dropout(0.2))
model.add(Convolution1D(32, 3,activation="relu"))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(256,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(2,activation="softmax"))
## compille it here according to instructions

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 248, 128)          115328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 248, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 246, 64)           24640     
_________________________________________________________________
dropout_3 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 32)           6176      
_________________________________________________________________
dropout_4 (Dropout)          (None, 244, 32)           0     

In [22]:

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 82s 4ms/step - loss: 0.5042 - acc: 0.7239 - val_loss: 0.3131 - val_acc: 0.8722
Epoch 2/5
20000/20000 [==============================] - 83s 4ms/step - loss: 0.3132 - acc: 0.8692 - val_loss: 0.3289 - val_acc: 0.8610
Epoch 3/5
20000/20000 [==============================] - 82s 4ms/step - loss: 0.2761 - acc: 0.8860 - val_loss: 0.3593 - val_acc: 0.8348
Epoch 4/5
20000/20000 [==============================] - 82s 4ms/step - loss: 0.2420 - acc: 0.9013 - val_loss: 0.2871 - val_acc: 0.8834
Epoch 5/5
20000/20000 [==============================] - 83s 4ms/step - loss: 0.2022 - acc: 0.9195 - val_loss: 0.2823 - val_acc: 0.8870


# Part F

Model constructed : (5 points)

Test Accuracy Over 87.5%: (5 Points)

Bonus: Min(10, Square of (test_score - 88%))

Create your best model, use Validation score to judge your best model and check accuracy on test set


In [18]:
print('Build model...')

## implement model here

model = Sequential()
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[g_word_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

model.add(embedding_layer)
model.add(LSTM(128,recurrent_dropout=0.2))
model.add(Dropout(0.3))
model.add(Dense(128,activation="relu"))
model.add(Dense(2,activation="softmax"))

## compille it here according to instructions
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 258       
Total params: 37,512,318
Trainable params: 236,418
Non-trainable params: 37,275,900
_________________________________________________________________
Model Built


You can keep saving models with different names in model_name, 

so you can retrieve their weights again for testing, you dont have to retrain 
(You would have to initialize the model definition again).

In [19]:
wt_dir = "./weights/"
model_name = 'model_best'
early_stopping =EarlyStopping(monitor='val_acc', patience=2)
bst_model_path = wt_dir + model_name + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, monitor='val_acc', save_best_only=True, save_weights_only=True)

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=7,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True,
         callbacks=[early_stopping, model_checkpoint])



Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/7
20000/20000 [==============================] - 209s 10ms/step - loss: 0.4586 - acc: 0.7810 - val_loss: 0.3111 - val_acc: 0.8656
Epoch 2/7
20000/20000 [==============================] - 207s 10ms/step - loss: 0.2910 - acc: 0.8789 - val_loss: 0.2826 - val_acc: 0.8812
Epoch 3/7
20000/20000 [==============================] - 206s 10ms/step - loss: 0.2510 - acc: 0.8985 - val_loss: 0.2800 - val_acc: 0.8900
Epoch 4/7
20000/20000 [==============================] - 207s 10ms/step - loss: 0.2181 - acc: 0.9123 - val_loss: 0.2601 - val_acc: 0.8946
Epoch 5/7
20000/20000 [==============================] - 207s 10ms/step - loss: 0.1827 - acc: 0.9287 - val_loss: 0.2681 - val_acc: 0.8902
Epoch 6/7
20000/20000 [==============================] - 207s 10ms/step - loss: 0.1437 - acc: 0.9444 - val_loss: 0.2882 - val_acc: 0.8988
Epoch 7/7
20000/20000 [==============================] - 206s 10ms/step - loss: 0.1072 - acc: 0.9607 - val_loss: 0

If you plan on using Ensemble averaging, feel free to edit the code below or add multiple models.

Make sure they get saved and can be retrieved when executing serially.

In [20]:
model.load_weights(bst_model_path)
scores = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

25000/25000 [==============================] - 79s 3ms/step
Accuracy: 89.61%


# Part G

Explain how Dense, LSTM and Convolution Layers work.

Explain Relu, Dropout, and Softmax work.

Analyze the architectures you constructed, with the accuracies you achieved and the training time it took. 

What are some insights you gained with these experiments? 

(5 Points)


Dense:
     fully connected layers. Then every neuron in previous layer is input for every neuron in next layer. 
LSTM:
    LSTM is a kind of RNN. Memorize long term information and important information previous stages. 
Convolution Layers:
    Convolute input and pass output to the next layer.
Relu:
    relu(x) =   x if x > 0
              { 
                0 if x <= 0
    f(x)=max(0,x)
Dropout:
    Eliminate some nodes in neural network by some probability to avoid overfitting.
Softmax work:
    Given the prediction and calculate the probabilities for each class, f(x)=exp(x)/sum(exp(x))
Analysis:
    The loss reduced to 0.1072 and the training accuracy is 0.9607, validation loss is 0.3212 and accuracy is 0.8846. Then the testing accuracy is 89.8%.
Insights:
    LSTM keeps long-term information well. Dropout works well for avoiding overfitting.